In [13]:
from typing import Any

import torch
import torch.nn as nn
import torch.nn.functional as F
from pytorch_lightning.utilities.types import OptimizerLRScheduler, STEP_OUTPUT
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import pytorch_lightning as pl
from pytorch_lightning import Trainer

In [14]:
# Step 1: Dataset Preparation
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Download and load CIFAR-10 dataset
train_data = datasets.CIFAR10(root="data", train=True, download=True, transform=transform)
val_data = datasets.CIFAR10(root="data", train=False, download=True, transform=transform)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
val_loader = DataLoader(val_data, batch_size=64, shuffle=False)

In [15]:
# Step 2: Define the Image Classification Model
class ImageClassifier(pl.LightningModule):
    def __init__(self):
        super(ImageClassifier, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.fc1 = nn.Linear(64 * 6 * 6, 128)
        self.fc2 = nn.Linear(128, 10)
    
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = x.view(-1, 64 * 6 * 6)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
    
        # Step 3: Configure the Training Process
    def training_step(self, batch, batch_idx):
        inputs, labels = batch
        outputs = self(inputs)
        loss = F.nll_loss(outputs, labels)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        inputs, labels = batch
        outputs = self(inputs)
        val_loss = F.nll_loss(outputs, labels)
        self.log('val_loss', val_loss)
        return val_loss
    
    def configure_optimizers(self) -> OptimizerLRScheduler:        
        optimizer = torch.optim.Adam(self.parameters(), lr=0.001)
        scheduler = {
            'scheduler': torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1),
            'interval': 'epoch',
            'frequency': 1
        }
        return {'optimizer': optimizer, 'lr_scheduler': scheduler}
    
    def test_step(self, batch, batch_idx) -> STEP_OUTPUT:
        inputs, labels = batch
        outputs = self(inputs)
        test_loss = F.nll_loss(outputs, labels)
        self.log('val_loss', test_loss)
        return {'val_loss': test_loss}

In [16]:
# Step 4: Training and Validation
# Initialize the model
model = ImageClassifier()

# Create a Trainer object
trainer = Trainer(max_epochs=1, devices=1, accelerator="auto")

# Train the model
trainer.fit(model, train_loader, val_loader)

Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name  | Type   | Params | Mode 
-----------------------------------------
0 | conv1 | Conv2d | 896    | train
1 | conv2 | Conv2d | 18.5 K | train
2 | fc1   | Linear | 295 K  | train
3 | fc2   | Linear | 1.3 K  | train
-----------------------------------------
315 K     Trainable params
0         Non-trainable params
315 K     Total params
1.263     Total estimated model params size (MB)
4         Modules in train mode
0         Modules in eval mode


Sanity Checking DataLoader 0:  50%|█████     | 1/2 [00:00<00:00, 11.49it/s]

C:\Users\ChristiaanMeijer\anaconda3\envs\confusion\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


C:\Users\ChristiaanMeijer\anaconda3\envs\confusion\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Epoch 0: 100%|██████████| 782/782 [00:24<00:00, 31.40it/s, v_num=0]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 1: 100%|██████████| 782/782 [00:25<00:00, 31.22it/s, v_num=0]       
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 2: 100%|██████████| 782/782 [00:25<00:00, 31.22it/s, v_num=0]       
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 3: 100%|██████████| 782/782 [00:26<00:00, 29.33it/s, v_num=0]       
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 4: 100%|██████████| 782/782 [00:28<00:00, 27.69it/s, v_num=0]       
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 5: 100%|██████████| 782/782 [00:28<00:00, 27.38it/s, v_num=0]       
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 6: 100%|██████████| 782/782 [00:32<00:00, 24.22it/s, v_num=0]       
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 7: 100%|██████████| 782/782 [00:31<00:00, 24.95it/s, v_num=0]       
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 8: 100%|██████████| 782/782 [00:3

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 782/782 [00:37<00:00, 20.99it/s, v_num=0]


In [19]:
model.test_step = model.validation_step

In [20]:
# Step 5: Testing
# Test the model
trainer.test(model, val_loader)

# Save the model (Optional)
trainer.save_checkpoint("image_classifier.ckpt")

# Load the model (Optional)
model = ImageClassifier.load_from_checkpoint("image_classifier.ckpt")

C:\Users\ChristiaanMeijer\anaconda3\envs\confusion\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 157/157 [00:03<00:00, 44.18it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        val_loss            1.0487003326416016
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [21]:
model.forward(val_loader.dataset[0][0].unsqueeze(0))  # Test the model with a single image

tensor([[-4.4999, -7.5618, -2.7844, -0.4533, -4.6251, -1.5486, -3.0632, -4.8176,
         -4.3754, -6.6145]], grad_fn=<LogSoftmaxBackward0>)

In [35]:
preds = [model(instance) for instance, label in val_loader.dataset]

In [58]:
# Get the predicted class indices and convert them to labels
predicted_labels = [torch.argmax(pred, dim=1).item() for pred in preds]
confidence_score = [torch.max(pred, dim=1).values.item() for pred in preds]
truths = [label for instance, label in val_loader.dataset]

In [67]:
data_csv_str = '\n'.join([','.join([str(e) for e in data_point]) for data_point in zip([str(i) for i in range(len(truths))],predicted_labels, truths, confidence_score, truths, truths)])
data_js_str = "const csvContent = `\nid,predicted,actual,confidence_score,processing_time_ms,image_size_kb\n" + data_csv_str + "`;\n"
# Save the data to a JavaScript file
with open('data_cifar10.js', 'w') as f:
    f.write(data_js_str)